In [62]:
import pyspark
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession

#In short all the funcitons in mllib library only work on RDDs
from pyspark.mllib.feature import StandardScaler,PCA #Only applicable to RDDs, therefore have to convert the DF in RDD 
from pyspark.mllib.stat import Statistics #Can only be applied on RDDs

#As here we are using ml library therefore, this function can be applied directly to DFs
from pyspark.ml.feature import VectorAssembler

In [63]:
import findspark
findspark.init()

In [64]:
spark = SparkSession.builder \
.master("local") \
.config("spark.driver.extraClassPath","C:/Users/AnshumaanChauhan/Documents/spark-3.3.0-bin-hadoop3/spark-3.3.0-bin-hadoop3/jars/mysql-connector-java-5.1.48.jar") \
.appName("Scalability Check of Systems for ML applications") \
.getOrCreate()

In [65]:
from pyspark.sql.functions import col, count, isnan, when

In [83]:
dataset = spark.read.csv('C:\\Users\AnshumaanChauhan\\Documents\\Systems for DS Umass\\Project\\archive (5)\\DelayedFlights.csv',
                         header=True)

In [67]:
#Checking which column has null or nan values how many times 
dataset.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dataset.columns]).show(vertical=True)

-RECORD 0-------------------
 _c0               | 0      
 Year              | 0      
 Month             | 0      
 DayofMonth        | 0      
 DayOfWeek         | 0      
 DepTime           | 0      
 CRSDepTime        | 0      
 ArrTime           | 7110   
 CRSArrTime        | 0      
 UniqueCarrier     | 0      
 FlightNum         | 0      
 TailNum           | 5      
 ActualElapsedTime | 8387   
 CRSElapsedTime    | 198    
 AirTime           | 8387   
 ArrDelay          | 8387   
 DepDelay          | 0      
 Origin            | 0      
 Dest              | 0      
 Distance          | 0      
 TaxiIn            | 7110   
 TaxiOut           | 455    
 Cancelled         | 0      
 CancellationCode  | 0      
 Diverted          | 0      
 CarrierDelay      | 689270 
 WeatherDelay      | 689270 
 NASDelay          | 689270 
 SecurityDelay     | 689270 
 LateAircraftDelay | 689270 



In [68]:
dataset.select(col("ArrDelay")).where((col("ArrDelay").isNull() | isnan(col("ArrDelay"))) & (col('Diverted')==1)).count()

7754

For all the flights that are diverted their ActualElaspedTime, ArrDelay and AirTime is null, because they did not land at correct destination

In [69]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("TaxiIn").isNull() | isnan(col("TaxiIn")))).count()

7110

In [70]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("TaxiIn").isNull() | isnan(col("TaxiIn"))) & (col("Diverted")==0)).count()

633

In [71]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("TaxiIn").isNull() | isnan(col("TaxiIn"))) & (col("Diverted")==0) & (col("CancellationCode")=='N')).count()

0

As we can infer above the TaxiIn are ArrTime are Null a few times. But it is the case when the flight is either cancelled or diverted. So it is fine if we not exclude the null values

In [72]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("TaxiOut").isNull() | isnan(col("TaxiOut")))).show(vertical=True)

-RECORD 0--------------------
 _c0               | 5484245 
 Year              | 2008    
 Month             | 10      
 DayofMonth        | 25      
 DayOfWeek         | 6       
 DepTime           | 1323.0  
 CRSDepTime        | 1255    
 ArrTime           | null    
 CRSArrTime        | 1442    
 UniqueCarrier     | XE      
 FlightNum         | 2347    
 TailNum           | N26549  
 ActualElapsedTime | null    
 CRSElapsedTime    | 107.0   
 AirTime           | null    
 ArrDelay          | null    
 DepDelay          | 28.0    
 Origin            | CLT     
 Dest              | EWR     
 Distance          | 529     
 TaxiIn            | null    
 TaxiOut           | null    
 Cancelled         | 1       
 CancellationCode  | B       
 Diverted          | 0       
 CarrierDelay      | null    
 WeatherDelay      | null    
 NASDelay          | null    
 SecurityDelay     | null    
 LateAircraftDelay | null    
-RECORD 1--------------------
 _c0               | 5486876 
 Year     

In [73]:
dataset.select(*(col(c) for c in dataset.columns)).where(((col("TaxiOut").isNull() | isnan(col("TaxiOut")))) & (col("CancellationCode")=='N')).show(vertical=True)

(0 rows)



We can infer from the above queries that TaxiOut is null only in the case of a cancelled flight. So it is fine if these are converted to 0s after type casting

In [74]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("CRSElapsedTime").isNull() | isnan(col("CRSElapsedTime")))).show(vertical=True)

-RECORD 0-------------------
 _c0               | 454268 
 Year              | 2008   
 Month             | 1      
 DayofMonth        | 31     
 DayOfWeek         | 4      
 DepTime           | 745.0  
 CRSDepTime        | 715    
 ArrTime           | null   
 CRSArrTime        | 941    
 UniqueCarrier     | 9E     
 FlightNum         | 2001   
 TailNum           | 91469E 
 ActualElapsedTime | null   
 CRSElapsedTime    | null   
 AirTime           | null   
 ArrDelay          | null   
 DepDelay          | 30.0   
 Origin            | ATL    
 Dest              | AUS    
 Distance          | 813    
 TaxiIn            | null   
 TaxiOut           | 15.0   
 Cancelled         | 0      
 CancellationCode  | N      
 Diverted          | 1      
 CarrierDelay      | null   
 WeatherDelay      | null   
 NASDelay          | null   
 SecurityDelay     | null   
 LateAircraftDelay | null   
-RECORD 1-------------------
 _c0               | 454394 
 Year              | 2008   
 Month        

In [75]:
dataset.select(*(col(c) for c in dataset.columns)).where((col("CRSElapsedTime").isNull() | isnan(col("CRSElapsedTime"))) & (col("Diverted")==0)).count()

0

We can infer from the above queries that CRSElapsedTime is null only in the case of a diverted flight. So it is fine if these are converted to 0s after type casting

Last columns have null values if the flight is cancelled instead of getting delayed, we do not drop these columns. Instead when we type cast the column to numeric data type these will be converted to zero which makes sense

In [76]:
dataset.count()

1936758

In [82]:
dataset.select("_c0").distinct().count()

1936758

Means that _c0 column is just for indexing the entreis, therefore we will use this column for joining the 2 datasets

In [85]:
#Have to do some pre processing here as otherwise everything will be sent as Text there 
list_of_columns=dataset.columns
Categorical_columns=['UniqueCarrier','TailNum','Origin','Dest','CancellationCode']
#Because rest everything in minutes or numerical, last columns that are delays for specific reasons are null if the flight is cancelled
#They will be converted to 0 when we do the conversion from string to numeric 

for col_name in Categorical_columns:
    list_of_columns.remove(col_name) 

#Null values will be converted to 0.0
#Adding _c0 for the join operation, so that we have the correct join, and no duplicates are created due to the change of null to 0.0
Categorical_columns.append("_c0")

numeric_dataset= dataset.select(*(col(c).cast('float') for c in list_of_columns))
updated_dataset= numeric_dataset.join(dataset.select(*(col(c) for c in Categorical_columns)),"_c0")

As the null values are handled during the pre-processing step we do not need to treat them by replacing them with the mean value, or some other technique used for handling the null values. Can also use pyspark.ml.features.Imputer module if have dataset which contains null values. (Best case is drop those rows if not facing with the problem of data insufficiency)

In [87]:
updated_dataset.printSchema()

root
 |-- _c0: float (nullable = true)
 |-- Year: float (nullable = true)
 |-- Month: float (nullable = true)
 |-- DayofMonth: float (nullable = true)
 |-- DayOfWeek: float (nullable = true)
 |-- DepTime: float (nullable = true)
 |-- CRSDepTime: float (nullable = true)
 |-- ArrTime: float (nullable = true)
 |-- CRSArrTime: float (nullable = true)
 |-- FlightNum: float (nullable = true)
 |-- ActualElapsedTime: float (nullable = true)
 |-- CRSElapsedTime: float (nullable = true)
 |-- AirTime: float (nullable = true)
 |-- ArrDelay: float (nullable = true)
 |-- DepDelay: float (nullable = true)
 |-- Distance: float (nullable = true)
 |-- TaxiIn: float (nullable = true)
 |-- TaxiOut: float (nullable = true)
 |-- Cancelled: float (nullable = true)
 |-- Diverted: float (nullable = true)
 |-- CarrierDelay: float (nullable = true)
 |-- WeatherDelay: float (nullable = true)
 |-- NASDelay: float (nullable = true)
 |-- SecurityDelay: float (nullable = true)
 |-- LateAircraftDelay: float (nullable 

In [89]:
#Loading dataset into MySQL 
updated_dataset.select(*(col(c) for c in dataset.columns)).write.format("jdbc") \
.option("url", "jdbc:mysql://localhost:3306/Sys") \
.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "dataset") \
.option("user", "root").option("password", "MySQL").save()

In [16]:
#Loading dataset from MySQL (after doing simple anaysis in MySQL)
queried_dataset = spark.read.format("jdbc") \
.option("url", "jdbc:mysql://localhost:3306/Sys") \
.option("driver", "com.mysql.jdbc.Driver").option("dbtable", "dataset") \
.option("user", "root").option("password", "MySQL").load()

In [6]:
queried_dataset.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (

For a better formatted description we could use toPandas() function 

In [11]:
queried_dataset.describe().toPandas()

,summary,_c0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,count,1936758,1936758,1936758,1936758,1936758,1936758,1936758,1929648,1936758,...,1929648,1936303,1936758,1936758,1936758,1247488,1247488,1247488,1247488,1247488
1,mean,3341651.1513358923,2008.0,6.11110629206127,15.753470490376186,3.9848272215733718,1518.5341168075722,1467.4726439751378,1610.140628757162,1634.224640868916,...,6.812975216205236,18.232202811233574,3.2683484462178546E-4,None,0.004003597764924684,19.179398920069772,3.7035706956700185,15.021635478657911,0.09013713959573158,25.296466178432176
2,stddev,2066064.9575203708,0.0,3.4825463936658796,8.776272060384269,1.9959662750518288,450.4852547937541,424.7667995772775,548.1781425365665,464.6347119906777,...,5.273595064525683,14.338534198201899,0.018075624257674595,None,0.06314721711010828,43.546207245474356,21.49290048410928,33.833052163653946,2.02271404750559,42.0548615209541
3,min,0,2008,1,1,1,1.0,0,1.0,0,...,0.0,0.0,0,A,0,0.0,0.0,0.0,0.0,0.0
4,max,999994,2008,9,9,7,959.0,959,959.0,959,...,99.0,99.0,1,N,1,997.0,996.0,992.0,99.0,994.0


In [ ]:
#Can convert this Dataframe into RDD 
queried_dataset_rdd= queried_dataset.rdd

Can see that it is showing everything in form Row(.....),Row(...) and so on. Here we cannot apply any function on the given data, so we need to get the actual data from the RDD, which can be done using the map function

In [ ]:
dataset_rdd= queried_dataset_rdd.map(lambda row:row[0:])

In [ ]:
dataset_rdd.collect()

In [ ]:
#Checking the relationship between variables, that is correlation and independence 
pearsonCorrMatrix = Statistics.corr(dataset_rdd,method='pearson')
#To visualize this better we will use Pandas dataframe 
DF_Corr= pd.DataFrame(pearsonCorrMatrix)

In [ ]:
#Convert categorical features into numerical columns 

In [ ]:
#Drop the redundant features - Diverted and Cancellation
dataset_rdd.drop([''])

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
fig = plt.figure(figsize=(15,15))

In [ ]:
#Standard Scaling

features_to_be_scaled = dataset_rdd.drop(['List of features to be not scaled'])
scaler= StandardScaler().fit(features_to_be_scaled)
scaled_features = scaler.transform(features_to_be_scaled)

In [ ]:
scaled_features.collect()

In [ ]:
#Vector Assembler will create a list for each entry in the RDD
#We will here split the dataset into X and Y, where Y will be the label whether the flight was delayed/cancelled/diverted

input_features= dataset_rdd.drop('label')

#Output of a vector assembler is a single column therefore outputCol instead of outputCols
#Initialzing the VectorAssembler
assembler= VectorAssembler(inputCols=input_features.columns,outputCol="features")

final_dataset = assembler.transform(dataset_rdd)

In [ ]:
final_dataset.select("features","label").show(truncated=False)

In [ ]:
splits = final_dataset.randomSplit([0.7, 0.3])
train_data = splits[0]
test_data = splits[1]

In [ ]:
# X=final_dataset.select('features')
# Y=final_dataset.select('label')

In [ ]:
# #Principal Component Analysis

# Pca=PCA(k=4)
# #K should be less than the number of features 
# pca_model = pca.fit(X)
# result= pca_model.transform(X)

In [ ]:
# result.collect()

Can see that the result is in form of DenseVectors and we need to convert it in DataFrames

In [ ]:
# train_data = result.map(lambda x: (x, )).toDF(["PCA Components"])

In [ ]:
# train_data.show(truncate=False)

In [ ]:
#Training Linear Regression
from pyspark.ml.regression import LinearRegression
lr_classifier = LinearRegression(featuresCol='features', labelCol='label')
lr_model=lr_classifier.fit(train_data)

In [ ]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

In [ ]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

In [ ]:
test_result = lr_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)